In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

NUM_OF_ROWS = 7500
ATTACK_NAME = 'DoS'

In [3]:
pd.set_option('display.max_columns', None)  # Show all columns

---

In [4]:
# import the attack sample dataset
dosSamples = pd.read_csv('dos_goldeneye_samples.csv') 
dosSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,3,515.344252,50,1282,462.681723,214074.376414,1402323,1248,535.646677,18,477.240416,266,154.531034,16,84.645472,0.0,0.0,590.451789,0,0,0,40.210026,75.926337,0.802339,0.013175,0.035396
1,1,115.500000,100,131,15.500000,240.250000,246,77,61.500000,46,15.500000,0,0.000000,0,0.000000,0.0,0.0,123.000000,0,0,0,8.514560,0.469784,8.514545,2.838187,4.013792
2,0,115.000000,115,115,0.000000,0.000000,0,0,0.000000,0,0.000000,81,81.000000,81,0.000000,0.0,0.0,0.000000,0,0,0,39.116539,0.997021,2.048092,1.029383,0.172540
3,1,98.500000,94,103,4.500000,20.250000,840,40,40.000000,40,0.000000,49,49.000000,49,0.000000,0.0,0.0,42.000000,0,0,0,37.484924,1.120450,2.009573,0.914266,0.938558
4,0,101.000000,101,101,0.000000,0.000000,0,0,0.000000,0,0.000000,47,47.000000,47,0.000000,0.0,0.0,0.000000,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,1,143.000000,143,143,0.000000,0.000000,218,109,109.000000,109,0.000000,0,0.000000,0,0.000000,0.0,0.0,218.000000,0,0,0,8.499306,0.235313,8.499306,8.499306,0.000000
324,2,98.166667,93,103,4.844814,23.472222,118,40,39.333333,39,0.471405,49,49.000000,49,0.000000,0.0,0.0,59.000000,0,0,0,8.036513,0.746592,6.632294,1.607303,2.538397
325,1,160.500000,66,548,174.403507,30416.583333,612,514,204.000000,32,219.642133,83,49.000000,32,24.041631,172.0,17.0,0.000000,0,6,0,0.163065,36.795153,0.066440,0.032613,0.029020
326,1,127.500000,92,163,35.500000,1260.250000,174,58,58.000000,58,0.000000,129,129.000000,129,0.000000,0.0,0.0,0.000000,0,0,0,0.005676,1057.076658,0.004388,0.001135,0.001652


In [5]:
dosSamples = dosSamples[dosSamples['ACK Flag Count'] > 100]

In [6]:
# print some general information about the attack samples
print(f'Dataset Shape: {dosSamples.shape}')
dosSamples.reset_index() 

Dataset Shape: (21, 26)


,index,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,26,1,124.775482,66,845,145.078895,21047.885732,584231,811,90.775482,32,145.078895,0,0.0,0,0.0,55.619484,0.0,117.268366,2539,3897,0,5.743676,1120.536707,0.023122,0.000893,0.001892
1,27,1,121.000527,66,849,140.240047,19667.270800,825026,815,87.000527,32,140.240047,0,0.0,0,0.0,52.001476,0.0,97.084726,3555,5928,0,10.782026,879.519319,0.029030,0.001137,0.002449
2,40,1,122.496462,66,860,141.830236,20115.815856,850451,826,88.496462,32,141.830236,0,0.0,0,0.0,53.694381,0.0,96.074446,3366,6244,0,11.796613,814.640611,0.078117,0.001228,0.002738
3,51,1,122.480506,66,829,142.044040,20176.509262,839680,795,88.480506,32,142.044040,0,0.0,0,0.0,53.649737,0.0,95.083230,3358,6132,0,12.465338,761.311086,0.051932,0.001314,0.002873
4,69,1,123.573296,66,831,143.762818,20667.747896,863397,797,89.573296,32,143.762818,0,0.0,0,0.0,54.859322,0.0,96.221665,3270,6369,0,12.742510,756.444383,0.061831,0.001322,0.002859
5,82,1,123.609547,66,881,143.952884,20722.432703,856040,847,89.609547,32,143.952884,0,0.0,0,0.0,54.859416,0.0,97.199955,3284,6269,0,14.330901,666.601486,0.063931,0.001500,0.003420
6,91,1,104.405674,54,833,120.419451,14500.844118,672515,799,70.405674,20,120.419451,0,0.0,0,0.0,36.545959,0.0,74.607832,2519,6834,3074,11.847213,806.265573,3.898364,0.001240,0.040068
7,106,1,109.580024,54,792,126.620150,16032.662369,690877,758,75.580024,20,126.620150,0,0.0,0,0.0,41.594246,0.0,82.345292,2911,5802,1722,14.243538,641.764701,5.236684,0.001558,0.054854
8,124,1,122.578599,66,813,141.961046,20152.938582,802965,779,88.578599,32,141.961046,0,0.0,0,0.0,53.393602,0.0,105.932058,3609,5456,0,8.945775,1013.327517,0.137966,0.000987,0.003047
9,136,1,106.246133,54,911,122.568377,15023.007152,541846,877,72.246133,20,122.568377,0,0.0,0,0.0,38.640800,0.0,81.005531,1871,5385,1990,7.606195,986.038358,0.115753,0.001014,0.003427


In [7]:
# find the columns that we need to synthesis data for to produce an attack dataset
columnsToGather = dosSamples.replace(0, np.nan) #replace all 0 values with null
columnsToGather = columnsToGather.dropna(how="all", axis=1).columns.tolist() #remove all columns where there are null values
columnsToGather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Packet Length Std',
 'Packet Length Variance',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Fwd Packet Length Std',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'ACK Flag Count',
 'RST Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

In [8]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
RST_FlagCount = dosSamples[dosSamples['RST Flag Count'] != 0]['RST Flag Count']

MinMaxDict = {col: (dosSamples[col].min() * 0.85, dosSamples[col].max() * 1.15) for col in columnsToGather}
MinMaxDict['Number of Ports'] = (1, 1)
MinMaxDict['RST Flag Count'] = (np.min(RST_FlagCount), np.max(RST_FlagCount))
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Length': (np.float64(88.74482307370185),
  np.float64(146.11285425101212)),
 'Packet Length Min': (np.float64(45.9), np.float64(75.89999999999999)),
 'Packet Length Max': (np.float64(673.1999999999999),
  np.float64(1047.6499999999999)),
 'Packet Length Std': (np.float64(102.35653313322236),
  np.float64(169.5604910365803)),
 'Packet Length Variance': (np.float64(12325.7175000617),
  np.float64(25000.66097440541)),
 'Total Length of Fwd Packet': (np.float64(378120.8),
  np.float64(1004422.6499999999)),
 'Fwd Packet Length Max': (np.float64(644.3), np.float64(1008.55)),
 'Fwd Packet Length Mean': (np.float64(59.84482307370184),
  np.float64(107.01285425101214)),
 'Fwd Packet Length Min': (np.float64(17.0), np.float64(36.8)),
 'Fwd Packet Length Std': (np.float64(102.35653313322236),
  np.float64(169.5604910365803)),
 'Fwd Segment Size Avg': (np.float64(31.06406511725293),
  np.float64(67.50656829320263)),
 'Subflow Fwd Bytes': (np.float64(63.

In [9]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['SYN Flag Count', 'ACK Flag Count', 'RST Flag Count', 'Packet Length Min', 'Packet Length Max', 'Total Length of Fwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Bwd Packet Length Max', 'Bwd Packet Length Min']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Length': (np.float64(88.74482307370185),
  np.float64(146.11285425101212)),
 'Packet Length Min': (45, 75),
 'Packet Length Max': (673, 1047),
 'Packet Length Std': (np.float64(102.35653313322236),
  np.float64(169.5604910365803)),
 'Packet Length Variance': (np.float64(12325.7175000617),
  np.float64(25000.66097440541)),
 'Total Length of Fwd Packet': (378120, 1004422),
 'Fwd Packet Length Max': (644, 1008),
 'Fwd Packet Length Mean': (np.float64(59.84482307370184),
  np.float64(107.01285425101214)),
 'Fwd Packet Length Min': (17, 36),
 'Fwd Packet Length Std': (np.float64(102.35653313322236),
  np.float64(169.5604910365803)),
 'Fwd Segment Size Avg': (np.float64(31.06406511725293),
  np.float64(67.50656829320263)),
 'Subflow Fwd Bytes': (np.float64(63.41665742178833),
  np.float64(134.85862103572862)),
 'SYN Flag Count': (1489, 4242),
 'ACK Flag Count': (3211, 7859),
 'RST Flag Count': (95, 3074),
 'Flow Duration': (np.float64(4.8821247577

### Creating the dataset

In [10]:
# creating an empty dataframe before adding values to it
dosDataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dosSamples.columns))), columns=dosSamples.columns)
dosDataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# adding zeros to all columns that should not have any values
zeroColumns = [col for col in dosSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    dosDataset[col] = int(0)
zeroColumns

['Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg']

In [12]:
dosDataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### First group

In [13]:
# finding the correlation between the SYN Flag Count column to the rest of the columns in order to create new data
firstCorrelation = ['SYN Flag Count', 'ACK Flag Count']
independent_col = dosSamples[firstCorrelation[0]].values.reshape(-1, 1) #column 'SYN Flag Count'
dependent_cols = dosSamples[firstCorrelation[1]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(firstCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('ACK Flag Count', np.float64(1.8289174466546883))


In [14]:
dosDataset['SYN Flag Count'] = np.random.randint(MinMaxDict['SYN Flag Count'][0], MinMaxDict['SYN Flag Count'][1], NUM_OF_ROWS)

for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values
        # calculate the value we want to add into the dataset in the given row
        delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
        updatedFactor = factor + random.choice([-1, 1]) * delta
        dosDataset.loc[index, col] = int(row['SYN Flag Count'] * updatedFactor)

In [15]:
dosDataset.head(10)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3417,5375.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4010,8664.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2962,4660.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2504,5437.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1964,3034.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3113,5041.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2282,4852.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4161,8971.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1879,4118.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3460,6984.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# adding the RST flag such that half of the rows will have 0 and the other half will have a random value from the known range
midpoint = len(dosDataset) // 2 #calculate the midpoint of the DataFrame

# Generate random integers for the second half
random_values = np.random.randint(MinMaxDict['RST Flag Count'][0], MinMaxDict['RST Flag Count'][1] + 1, size=len(dosDataset) - midpoint)

# Add the new column with 0s for the first half and random integers for the second half
new_column_values = [0] * midpoint + list(random_values)
dosDataset['RST Flag Count'] = new_column_values

In [17]:
dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3417,5375.0,0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4010,8664.0,0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2962,4660.0,0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2504,5437.0,0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1964,3034.0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3690,5593.0,2401,0.0,0.0,0.0,0.0,0.0
7496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3844,8148.0,2538,0.0,0.0,0.0,0.0,0.0
7497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2381,3712.0,3059,0.0,0.0,0.0,0.0,0.0
7498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3169,6825.0,316,0.0,0.0,0.0,0.0,0.0


### Second group

In [18]:
# Generate random values for the 'Flow Duration' column
randValues = np.random.uniform(MinMaxDict['Flow Duration'][0]*0.9, MinMaxDict['Flow Duration'][1]*1.05, size=NUM_OF_ROWS)

# Assign the random values
dosDataset['Flow Duration'] = randValues

In [19]:
dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3417,5375.0,0,40.517026,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4010,8664.0,0,12.107488,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2962,4660.0,0,7.713602,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2504,5437.0,0,22.638434,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1964,3034.0,0,25.542708,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3690,5593.0,2401,32.899128,0.0,0.0,0.0,0.0
7496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3844,8148.0,2538,22.397644,0.0,0.0,0.0,0.0
7497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2381,3712.0,3059,14.561753,0.0,0.0,0.0,0.0
7498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3169,6825.0,316,20.296617,0.0,0.0,0.0,0.0


In [20]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
secondCorrelation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dosSamples[secondCorrelation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = dosSamples[secondCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(secondCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packets Per Second', np.float64(30.145241232971678))
('IAT Max', np.float64(0.3696369152684689))
('IAT Mean', np.float64(0.00011623768083943193))
('IAT Std', np.float64(0.0041811844462341175))


In [21]:
durationToPacketsCorr = [x * y for x, y in zip(dosSamples['Flow Duration'].values, dosSamples['Packets Per Second'].values)]
durationToPacketsCorr = np.mean(durationToPacketsCorr)
durationToPacketsCorr

np.float64(8732.047619047618)

In [22]:
#iterating over all rows we need to add values
for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: 
        # calculate a random small delta of the factor for adding some randomness
        if col == 'Packets Per Second':
            delta = random.uniform(durationToPacketsCorr * 0.1, durationToPacketsCorr * 0.2) # select a delta
            updatedFactor = durationToPacketsCorr + random.choice([-1, 1]) * delta
            dosDataset.loc[index, col] = updatedFactor / row['Flow Duration']
        else:
            if col == 'IAT Std' or col == 'IAT Max':
                delta = random.uniform(factor * 0.7, factor * 0.99)
                updatedFactor = factor + random.choices([-1, 1], weights=[2, 1], k=1)[0] * delta  
            else:
                delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
                updatedFactor = factor + random.choice([-1, 1]) * delta
            dosDataset.loc[index, col] = row['Flow Duration'] * updatedFactor

In [23]:
dosSamples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,21.0,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.0,21.0,21.0,21.0,21.000000,21.0,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
mean,1.0,118.162351,62.000000,835.333333,137.140846,18874.235571,737972.047619,801.333333,84.162351,28.000000,137.140846,0.0,0.0,0.0,0.0,49.548964,0.0,95.038656,3010.761905,5627.904762,562.238095,15.224132,700.470398,4.264842,0.001767,0.049179
std,0.0,6.975820,5.796551,29.169048,8.363914,2241.856330,128170.354837,29.169048,6.975820,5.796551,8.363914,0.0,0.0,0.0,0.0,6.637618,0.0,9.910132,551.209389,852.917400,897.206103,7.796555,276.305968,6.595806,0.000968,0.074095
min,1.0,104.405674,54.000000,792.000000,120.419451,14500.844118,444848.000000,758.000000,70.405674,20.000000,120.419451,0.0,0.0,0.0,0.0,36.545959,0.0,74.607832,1752.000000,3778.000000,0.000000,5.743676,246.263767,0.023122,0.000893,0.001892
25%,1.0,109.976270,54.000000,818.000000,127.842838,16343.791319,640328.000000,784.000000,75.976270,20.000000,127.842838,0.0,0.0,0.0,0.0,41.808258,0.0,88.539871,2656.000000,5385.000000,0.000000,9.452635,457.067517,0.063931,0.001061,0.002862
50%,1.0,121.000527,66.000000,831.000000,140.240047,19667.270800,773385.000000,797.000000,87.000527,32.000000,140.240047,0.0,0.0,0.0,0.0,52.030510,0.0,96.074446,3234.000000,5802.000000,0.000000,12.632054,756.444383,0.137966,0.001322,0.003427
75%,1.0,123.044975,66.000000,849.000000,142.734482,20373.132473,846118.000000,815.000000,89.044975,32.000000,142.734482,0.0,0.0,0.0,0.0,53.888530,0.0,101.706229,3366.000000,6244.000000,1035.000000,19.535407,943.017495,5.236684,0.002188,0.054854
max,1.0,127.054656,66.000000,911.000000,147.443905,21739.705195,873411.000000,877.000000,93.054656,32.000000,147.443905,0.0,0.0,0.0,0.0,58.701364,0.0,117.268366,3689.000000,6834.000000,3074.000000,34.094212,1120.536707,20.601452,0.004061,0.230153


In [24]:
dosDataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.000000,7500.000000,7500.000000,7500.000000,7500.000000,7500.000000,7500.000000,7500.000000
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2883.574933,5254.780800,793.787467,22.575559,535.617215,6.028531,0.002615,0.068939
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,791.919877,1666.868578,1000.788734,10.531335,387.786137,7.953860,0.001294,0.089994
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1489.000000,2182.000000,0.000000,4.394071,171.690284,0.022332,0.000413,0.000267
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2213.000000,3926.000000,0.000000,13.458077,274.418568,0.734565,0.001521,0.008533
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2890.500000,5143.000000,47.500000,22.557708,388.819654,1.877442,0.002561,0.021961
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3574.250000,6316.250000,1571.250000,31.582777,650.027314,9.358631,0.003575,0.108201
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4241.000000,9253.000000,3074.000000,41.168042,2309.052556,30.261367,0.005718,0.340094


In [25]:
x = dosDataset[dosDataset['Flow Duration']<6]
x[x['Flow Duration']>5][0:40]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2282,4852.0,0,5.964903,1628.229262,3.987792,0.000803,0.003641
102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2961,6378.0,0,5.629028,1395.900413,0.484905,0.000527,0.004747
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3353,6861.0,0,5.387072,1910.494187,0.254589,0.000730,0.044326
142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4205,6764.0,0,5.541786,1764.270473,0.405083,0.000539,0.041648
149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2196,4716.0,0,5.917311,1325.186626,0.457936,0.000808,0.000662
157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2575,3874.0,0,5.120700,1438.174293,0.128856,0.000524,0.002679
234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3308,5083.0,0,5.931474,1281.409426,4.079700,0.000619,0.005181
248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2423,3981.0,0,5.591851,1726.065949,0.261790,0.000560,0.043529
398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2853,5885.0,0,5.595353,1396.060083,0.512302,0.000747,0.002287
481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3626,5912.0,0,5.420973,1878.529450,0.383000,0.000709,0.004422


In [26]:
x = dosDataset[dosDataset['Flow Duration']<50.5]
x[x['Flow Duration']>25][20:40]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2111,4320.0,0,32.808011,235.907968,0.559143,0.004374,0.234716
41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1609,2385.0,0,35.605178,273.529598,3.082445,0.004796,0.043162
44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2612,5513.0,0,35.814072,210.675443,25.032252,0.003533,0.031812
45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3004,6145.0,0,38.219447,200.651921,26.409208,0.003640,0.041903
46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2959,6473.0,0,39.833010,258.480395,0.628079,0.003964,0.049380
49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2448,3770.0,0,26.022670,284.012226,2.505363,0.002557,0.028691
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1982,2987.0,0,37.070566,267.661884,2.222169,0.005132,0.280167
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4053,8366.0,0,35.367547,199.986043,0.175422,0.004871,0.042481
54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3795,7955.0,0,26.287823,367.940766,1.857660,0.002677,0.027593
57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1962,4263.0,0,30.714513,235.384424,0.472317,0.003937,0.237427


In [27]:
dosDataset[dosDataset['Flow Duration']<6][:20]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2282,4852.0,0,5.964903,1628.229262,3.987792,0.000803,0.003641
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2830,4555.0,0,4.809958,1625.131607,0.095664,0.000620,0.004381
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2570,5310.0,0,4.746819,1616.511283,0.144791,0.000627,0.002861
102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2961,6378.0,0,5.629028,1395.900413,0.484905,0.000527,0.004747
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3353,6861.0,0,5.387072,1910.494187,0.254589,0.000730,0.044326
122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1942,2899.0,0,4.557595,1672.584493,0.410425,0.000636,0.003124
135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1610,3519.0,0,4.916684,1511.568354,0.417829,0.000667,0.004923
142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4205,6764.0,0,5.541786,1764.270473,0.405083,0.000539,0.041648
149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2196,4716.0,0,5.917311,1325.186626,0.457936,0.000808,0.000662
150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3024,6256.0,0,4.492822,2145.678586,0.291880,0.000585,0.004862


### Third group

In [28]:
# finding the correlation between the Packet Length Std column to the rest of the columns in order to create new data
firstCorrelation = ['Average Packet Length', 'Packet Length Std', 'Packet Length Variance', 'Total Length of Fwd Packet', 
                    'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Fwd Segment Size Avg', 'Subflow Fwd Bytes']
independent_col = dosSamples[firstCorrelation[0]].values.reshape(-1, 1) #column 'Packet Length Std'
dependent_cols = dosSamples[firstCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(firstCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packet Length Std', np.float64(1.1607160816175794))
('Packet Length Variance', np.float64(160.25967924010962))
('Total Length of Fwd Packet', np.float64(6271.216232539372))
('Fwd Packet Length Mean', np.float64(0.7132122210543036))
('Fwd Packet Length Std', np.float64(1.1607160816175794))
('Fwd Segment Size Avg', np.float64(0.4210840873569066))
('Subflow Fwd Bytes', np.float64(0.8054518323891989))


In [29]:
dosDataset['Average Packet Length'] = np.random.uniform(MinMaxDict['Average Packet Length'][0]*0.85, MinMaxDict['Average Packet Length'][1]*1.15, NUM_OF_ROWS)

for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values
        # calculate the value we want to add into the dataset in the given row
        delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
        updatedFactor = factor + random.choice([-1, 1]) * delta
        dosDataset.loc[index, col] = row['Average Packet Length'] * updatedFactor

In [30]:
x = dosDataset[dosDataset['Packet Length Std'] > 114]
x[x['Packet Length Std'] < 120].tail(20)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
7117,0.0,120.393685,0.0,0.0,117.108386,16544.309344,628653.599851,0.0,98.429959,0.0,162.212245,0,0,0,0,56.070248,0,112.248726,2792,5736.0,2727,7.981382,1307.056185,0.499149,0.000767,0.000698
7193,0.0,84.371950,0.0,0.0,114.597284,15814.062711,614519.093329,0.0,67.240770,0.0,78.669164,0,0,0,0,31.181321,0,57.402677,3772,5549.0,1574,21.519945,351.493443,14.536939,0.002206,0.006232
7217,0.0,124.283841,0.0,0.0,119.870540,23207.855410,916276.402885,0.0,74.347363,0.0,116.878176,0,0,0,0,42.144774,0,118.873701,4063,6649.0,2396,31.653629,245.482503,3.087081,0.003243,0.025738
7281,0.0,84.434996,0.0,0.0,114.657630,15913.842192,620078.457635,0.0,71.834841,0.0,109.199683,0,0,0,0,31.892142,0,74.990423,3711,5738.0,1105,17.250690,417.509592,0.434591,0.002237,0.005088
7291,0.0,86.774930,0.0,0.0,119.283503,16324.006950,612900.442406,0.0,50.074199,0.0,80.929602,0,0,0,0,30.110917,0,61.475072,3086,4619.0,986,36.714870,211.521344,2.985817,0.003606,0.265935
7299,0.0,90.257499,0.0,0.0,116.484885,17089.773041,637889.986399,0.0,74.535738,0.0,124.188085,0,0,0,0,31.075038,0,85.510806,4069,6129.0,2665,15.169023,515.125691,10.955828,0.002067,0.008237
7301,0.0,117.169929,0.0,0.0,119.757566,21180.341312,865412.149450,0.0,71.534097,0.0,114.800464,0,0,0,0,40.220890,0,77.422868,4225,8884.0,152,10.036224,711.142653,7.244537,0.000975,0.009840
7305,0.0,120.933961,0.0,0.0,117.962520,16122.980048,872254.112329,0.0,98.654215,0.0,122.186943,0,0,0,0,41.562650,0,84.357118,3816,7765.0,1120,14.440411,522.482887,9.105941,0.002012,0.016035
7357,0.0,120.814108,0.0,0.0,116.992479,15565.026735,907486.347075,0.0,101.041117,0.0,124.565446,0,0,0,0,60.469653,0,80.477679,4026,6205.0,2322,9.044379,860.185871,0.724224,0.001213,0.005914
7363,0.0,114.189177,0.0,0.0,116.832697,16061.873886,619166.513874,0.0,69.148131,0.0,153.564156,0,0,0,0,39.048352,0,104.752267,3574,7788.0,1407,11.434936,852.744209,0.095252,0.001479,0.008522


### Independant Columns

In [31]:
# finding the correlation between the Packet Length Std column to the rest of the columns in order to create new data
independantColumns = ['Packet Length Min', 'Packet Length Max', 'Fwd Packet Length Max', 'Fwd Packet Length Min']

for col in independantColumns:
    dosDataset[col] = (np.random.uniform(MinMaxDict[col][0]*0.85, MinMaxDict[col][1]*1.15, NUM_OF_ROWS)).astype(int)

dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,160.414928,52,1076,216.424606,20613.863404,8.869285e+05,596,102.801018,24,215.525747,0,0,0,0,75.760240,0,143.864511,3417,5375.0,0,40.517026,238.336616,3.696926,0.004071,0.010993
1,0.0,104.675670,64,832,137.721529,18795.103005,5.702754e+05,560,63.807804,32,139.882878,0,0,0,0,52.141428,0,74.697613,4010,8664.0,0,12.107488,823.522253,0.060181,0.001635,0.006073
2,0.0,133.230477,56,1155,181.114535,17745.209245,6.947208e+05,1006,79.529092,40,138.288642,0,0,0,0,47.141016,0,86.220033,2962,4660.0,0,7.713602,1002.747182,4.875958,0.000719,0.002052
3,0.0,81.510306,83,896,78.150675,10934.094988,6.050432e+05,941,51.218438,15,78.546635,0,0,0,0,39.695277,0,74.175937,2504,5437.0,0,22.638434,445.654153,15.513614,0.002331,0.017301
4,0.0,149.394621,65,930,154.091396,20566.223089,7.770156e+05,668,122.333774,34,195.785132,0,0,0,0,53.686911,0,133.616061,1964,3034.0,0,25.542708,294.472471,1.660594,0.002516,0.007626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.0,153.163230,39,1133,157.093916,21337.233796,1.101319e+06,566,128.146734,22,146.504395,0,0,0,0,53.245756,0,147.547373,3690,5593.0,2401,32.899128,223.623727,1.843407,0.004516,0.031142
7496,0.0,164.914656,79,699,211.905346,21790.100802,1.156787e+06,1009,140.392689,26,212.414500,0,0,0,0,81.881346,0,119.172219,3844,8148.0,2538,22.397644,349.847245,2.114874,0.002131,0.016004
7497,0.0,147.835872,44,1140,154.106965,21209.348172,8.127261e+05,861,91.613798,35,191.523858,0,0,0,0,53.404754,0,97.580085,2381,3712.0,3059,14.561753,538.987392,1.354595,0.001916,0.017420
7498,0.0,131.465816,41,741,136.859699,18362.001674,7.129807e+05,753,103.960111,17,134.049780,0,0,0,0,46.123383,0,93.153371,3169,6825.0,316,20.296617,369.083990,1.100066,0.002763,0.010310


### Adding labels and verifiying the dataset

In [32]:
# adding number of ports and a label to the dataset
dosDataset['Number of Ports'] = np.full(shape=NUM_OF_ROWS, fill_value=1, dtype=int)
dosDataset['Label'] = ATTACK_NAME

In [33]:
dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std,Label
0,1,160.414928,52,1076,216.424606,20613.863404,8.869285e+05,596,102.801018,24,215.525747,0,0,0,0,75.760240,0,143.864511,3417,5375.0,0,40.517026,238.336616,3.696926,0.004071,0.010993,DoS
1,1,104.675670,64,832,137.721529,18795.103005,5.702754e+05,560,63.807804,32,139.882878,0,0,0,0,52.141428,0,74.697613,4010,8664.0,0,12.107488,823.522253,0.060181,0.001635,0.006073,DoS
2,1,133.230477,56,1155,181.114535,17745.209245,6.947208e+05,1006,79.529092,40,138.288642,0,0,0,0,47.141016,0,86.220033,2962,4660.0,0,7.713602,1002.747182,4.875958,0.000719,0.002052,DoS
3,1,81.510306,83,896,78.150675,10934.094988,6.050432e+05,941,51.218438,15,78.546635,0,0,0,0,39.695277,0,74.175937,2504,5437.0,0,22.638434,445.654153,15.513614,0.002331,0.017301,DoS
4,1,149.394621,65,930,154.091396,20566.223089,7.770156e+05,668,122.333774,34,195.785132,0,0,0,0,53.686911,0,133.616061,1964,3034.0,0,25.542708,294.472471,1.660594,0.002516,0.007626,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,1,153.163230,39,1133,157.093916,21337.233796,1.101319e+06,566,128.146734,22,146.504395,0,0,0,0,53.245756,0,147.547373,3690,5593.0,2401,32.899128,223.623727,1.843407,0.004516,0.031142,DoS
7496,1,164.914656,79,699,211.905346,21790.100802,1.156787e+06,1009,140.392689,26,212.414500,0,0,0,0,81.881346,0,119.172219,3844,8148.0,2538,22.397644,349.847245,2.114874,0.002131,0.016004,DoS
7497,1,147.835872,44,1140,154.106965,21209.348172,8.127261e+05,861,91.613798,35,191.523858,0,0,0,0,53.404754,0,97.580085,2381,3712.0,3059,14.561753,538.987392,1.354595,0.001916,0.017420,DoS
7498,1,131.465816,41,741,136.859699,18362.001674,7.129807e+05,753,103.960111,17,134.049780,0,0,0,0,46.123383,0,93.153371,3169,6825.0,316,20.296617,369.083990,1.100066,0.002763,0.010310,DoS


In [34]:
dosSamples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,21.0,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.0,21.0,21.0,21.0,21.000000,21.0,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
mean,1.0,118.162351,62.000000,835.333333,137.140846,18874.235571,737972.047619,801.333333,84.162351,28.000000,137.140846,0.0,0.0,0.0,0.0,49.548964,0.0,95.038656,3010.761905,5627.904762,562.238095,15.224132,700.470398,4.264842,0.001767,0.049179
std,0.0,6.975820,5.796551,29.169048,8.363914,2241.856330,128170.354837,29.169048,6.975820,5.796551,8.363914,0.0,0.0,0.0,0.0,6.637618,0.0,9.910132,551.209389,852.917400,897.206103,7.796555,276.305968,6.595806,0.000968,0.074095
min,1.0,104.405674,54.000000,792.000000,120.419451,14500.844118,444848.000000,758.000000,70.405674,20.000000,120.419451,0.0,0.0,0.0,0.0,36.545959,0.0,74.607832,1752.000000,3778.000000,0.000000,5.743676,246.263767,0.023122,0.000893,0.001892
25%,1.0,109.976270,54.000000,818.000000,127.842838,16343.791319,640328.000000,784.000000,75.976270,20.000000,127.842838,0.0,0.0,0.0,0.0,41.808258,0.0,88.539871,2656.000000,5385.000000,0.000000,9.452635,457.067517,0.063931,0.001061,0.002862
50%,1.0,121.000527,66.000000,831.000000,140.240047,19667.270800,773385.000000,797.000000,87.000527,32.000000,140.240047,0.0,0.0,0.0,0.0,52.030510,0.0,96.074446,3234.000000,5802.000000,0.000000,12.632054,756.444383,0.137966,0.001322,0.003427
75%,1.0,123.044975,66.000000,849.000000,142.734482,20373.132473,846118.000000,815.000000,89.044975,32.000000,142.734482,0.0,0.0,0.0,0.0,53.888530,0.0,101.706229,3366.000000,6244.000000,1035.000000,19.535407,943.017495,5.236684,0.002188,0.054854
max,1.0,127.054656,66.000000,911.000000,147.443905,21739.705195,873411.000000,877.000000,93.054656,32.000000,147.443905,0.0,0.0,0.0,0.0,58.701364,0.0,117.268366,3689.000000,6834.000000,3074.000000,34.094212,1120.536707,20.601452,0.004061,0.230153


In [35]:
dosDataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,7500.0,7500.000000,7500.000000,7500.000000,7500.000000,7500.000000,7.500000e+03,7500.000000,7500.000000,7500.000000,7500.000000,7500.0,7500.0,7500.0,7500.0,7500.000000,7500.0,7500.000000,7500.000000,7500.000000,7500.000000,7500.000000,7500.000000,7500.000000,7500.000000,7500.000000
mean,1.0,121.947093,61.792800,885.043067,141.294440,19550.684552,7.655796e+05,852.636400,86.564188,27.462267,141.921888,0.0,0.0,0.0,0.0,51.333937,0.0,98.349190,2883.574933,5254.780800,793.787467,22.575559,535.617215,6.028531,0.002615,0.068939
std,0.0,26.714671,13.689351,182.677362,38.049922,5235.371420,2.059538e+05,176.283217,23.425815,7.767287,38.061452,0.0,0.0,0.0,0.0,13.789625,0.0,26.464404,791.919877,1666.868578,1000.788734,10.531335,387.786137,7.953860,0.001294,0.089994
min,1.0,75.438627,38.000000,572.000000,70.783110,9707.958169,3.829514e+05,547.000000,43.451920,14.000000,70.293800,0.0,0.0,0.0,0.0,25.433580,0.0,48.979504,1489.000000,2182.000000,0.000000,4.394071,171.690284,0.022332,0.000413,0.000267
25%,1.0,98.927797,50.000000,728.000000,111.622405,15508.112490,6.054793e+05,701.000000,68.374399,21.000000,112.300191,0.0,0.0,0.0,0.0,40.587097,0.0,77.504698,2213.000000,3926.000000,0.000000,13.458077,274.418568,0.734565,0.001521,0.008533
50%,1.0,121.716840,62.000000,886.000000,138.060274,19112.931487,7.507743e+05,849.000000,84.592790,28.000000,138.804764,0.0,0.0,0.0,0.0,50.105219,0.0,96.127355,2890.500000,5143.000000,47.500000,22.557708,388.819654,1.877442,0.002561,0.021961
75%,1.0,145.347479,74.000000,1044.000000,165.834641,22797.661700,8.968803e+05,1006.000000,101.396664,34.000000,165.758898,0.0,0.0,0.0,0.0,59.913646,0.0,115.397191,3574.250000,6316.250000,1571.250000,31.582777,650.027314,9.358631,0.003575,0.108201
max,1.0,168.015664,86.000000,1203.000000,233.667125,32213.778423,1.259497e+06,1159.000000,143.554691,41.000000,233.639195,0.0,0.0,0.0,0.0,84.366951,0.0,161.583704,4241.000000,9253.000000,3074.000000,41.168042,2309.052556,30.261367,0.005718,0.340094


In [ ]:
# save the dataset
# dosDataset.to_csv('dos_goldeneye_dataset_new.csv', index=False)

---